# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройтем файл с данными и изучим общую информацию

In [1]:
import pandas as pd

data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [3]:
for column in data:
    print('-----------------------------------')
    print('Уникальные значения колонки:', column)
    print('-----------------------------------')
    print(data[column].value_counts())

-----------------------------------
Уникальные значения колонки: children
-----------------------------------
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-----------------------------------
Уникальные значения колонки: days_employed
-----------------------------------
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
-----------------------------------
Уникальные значения колонки: dob_years
-----------------------------------
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57  

In [4]:
data.pop('days_employed')

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

**Вывод**

Очевидны пропуски в столбце доходов и стажа. В столбце дети есть артефакты (минус 1 ребенок или 20 детей выглядят очень странно). В столбце образования дублирующиеся данные в разных регистрах. В столбце назначение дублирующиеся значения но с разными формулировками. Столбцы семейного статуса и образования дублируются столбцами с индетификационным номером. Удобно для категоризации.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
total_income_median_group = data.groupby('income_type')['total_income'].transform('median')
print(total_income_median_group.unique())

data['total_income'] = data['total_income'].fillna(value = total_income_median_group)

data.info()

[142594.3968474  118514.48641164 172357.95096577 150447.93528301
 131339.75167621 499163.14494709  98201.62531401  53829.13072906]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

Пустых строк нет. Все количественные переменные в числовом типе - вещественные (float64) или целое число (int64). Т.к. при арифметических операциях значений с типами int и float получается значение типа float, а в будущем мы возможно будем осуществлять эти самые операции между доходом и идентификаторами, а так же что бы не захломлять таблицу длинными числовыми значениями - есть необходимость для замены типа данных методом *astype()* - вещественных в целое число в столбце *total_income*.

### Замена типа данных

In [6]:
data['total_income'] = data['total_income'].astype('int64')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


**Вывод**

Вещественного типа данных в нашей таблице больше нет. Но очевидно что в таблице много дублированных записей (хотя бы в идентификаторах), однако в нашем случае наиболее распространенный вариант - дубликаты в строковых данных с разным регистром (например "Среднее" и "среднее"). И только в столбце *purpose* одинаковые по смыслу категории с разными названиями ("на покупку своего автомобиля" и "на покупку автомобиля")

### Избавление от артефактов

In [7]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
print(data['children'].unique())

[1 0 3 2 4 5]


**Вывод**

Минус 1 ребенок явный артефакт, возникший скорее всего при переносе значений, при котором было скапиравно тире "-".
20 детей теоретически может быть правдой, но так как доля таких случаев минимальна в отношении к случаям с двумя детьми и огромна в отношении к случаям 5 детей, то примем значение в 20 детей за ошибку, возможно изначальное значение было 2.0

### Обработка дубликатов

In [8]:
print('Дубликатов в таблице до:',data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop= True)

print('Дубликатов в таблице после:',data.duplicated().sum())

Дубликатов в таблице до: 54
Дубликатов в таблице после: 0


In [9]:
for column in data:
    if data[column].dtype == object:
        data[column] = data[column].str.lower()
        print('-----------------------------------')
        print('Уникальные значения колонки:', column)
        print('-----------------------------------')
        print(data[column].value_counts())

-----------------------------------
Уникальные значения колонки: education
-----------------------------------
среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
-----------------------------------
Уникальные значения колонки: family_status
-----------------------------------
женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
-----------------------------------
Уникальные значения колонки: gender
-----------------------------------
f      14189
m       7281
xna        1
Name: gender, dtype: int64
-----------------------------------
Уникальные значения колонки: income_type
-----------------------------------
сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        

**Вывод**

Избавились от дубликатов в строковых данных с разным регистром. Однако вопрос с одинаковыми по смыслу категориями с разными названиями в столбце *purpose* остался

### Лемматизация

In [10]:
from pymystem3 import Mystem
m = Mystem()
unique_purposes = data['purpose'].unique()
print(unique_purposes)

lemmas=[]
def category(s):
    lemmas = m.lemmatize(s)
    for row in lemmas:
        if 'недвижимость' in row or 'жилье' in row:
            return 'недвижимость'
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'образование' in row:
            return 'образование'
        if 'свадьба' in row:
            return 'свадьба'

data['category'] = data['purpose'].apply(category)
print()
print()
print(data['category'].head(10))
print()
print()
print(data['category'].value_counts())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


0    недвижимость
1      ав

In [11]:
#data['cat'] = data['purpose_lemm']
#data.loc[data['cat'].str.contains('свадьба'), 'cat'] = 'свадьба'
#data.loc[data['cat'].str.contains('автомобиль'), 'cat'] = 'автомобиль'
#data.loc[data['cat'].str.contains('образование'), 'cat'] = 'образование'
#data.loc[data['cat'].str.contains('жилье'), 'cat'] = 'недвижимость'
#data.loc[data['cat'].str.contains('недвижимость'), 'cat'] = 'недвижимость'

#unique_purposes2 = data['cat'].unique()
#print(unique_purposes2)
#print('--------------------------------')
#print(data['cat'].value_counts())
#print('--------------------------------')
#print(data['cat'].head(10))

**Вывод**

Люди берут кредит чаще всего для покупки недвижимости. Реже всего на свадьбу.

### Категоризация данных

In [12]:
def children_status(children):
    if children == 0:
        return 'Нет детей'
    if children >= 2:
        return 'Многодетное домохозяйство'
    else:
        return 'Нуклеарное домохозяйство'
       
data['children_status'] = data['children'].apply(children_status)
print(data['children_status'].value_counts())

Нет детей                    14107
Нуклеарное домохозяйство      4856
Многодетное домохозяйство     2508
Name: children_status, dtype: int64


In [13]:
def family_status_cat (family_status_id):
    if family_status_id >= 2:
        return 'Одинокие'
    else:
        return 'Семейные'
       
data['family_status_cat'] = data['family_status_id'].apply(family_status_cat)
print(data['family_status_cat'].value_counts())

Семейные    16507
Одинокие     4964
Name: family_status_cat, dtype: int64


In [14]:
def income_status (total_income):
    if total_income <= 100000:
        return 'Бедные'
    if total_income >= 500000:
        return 'Очень богатые'
    else:
        return 'Богатые'
       
data['income_status'] = data['total_income'].apply(income_status)
print(data['income_status'].value_counts())

Богатые          16786
Бедные            4463
Очень богатые      222
Name: income_status, dtype: int64


**Вывод**

Среди заемщиков в основном преобладают бездетные семейные домохозяйства. Многодетных относительно мало. Людей с достатком свыше 100 тысяч но менее 500 тысяч на удивление много (больше бедных людей). Видимо бедные люди предпочитают не брать в долг или не брать в долг в банке.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
def children_debt(row):
    
    children_status = row['children_status']
    debt = row['debt']
    
    if children_status == 'Нет детей':
        if debt == 1:
                return 'Домохозяйство без детей с долгом'
        return 'Домохозяйство без детей и без долга'
    if children_status == 'Многодетное домохозяйство':
        if debt == 1:
                return 'Многодетное домохозяйство с долгом'
        return 'Многодетное домохозяйство без долга'
    if children_status == 'Нуклеарное домохозяйство':
        if debt == 1:
                return 'Нуклеарное домохозяйство с долгом'
            
    return 'Нуклеарное домохозяйство без долгов'

row_values = [data['children_status'], data['debt']]
row_columns = ['children', 'debt']
row = pd.Series(data=row_values , index=row_columns) 
data['status_child'] = data.apply(children_debt, axis=1)

print(data['status_child'].value_counts())
print('--------------------------------')
print(data['status_child'].unique())
print('--------------------------------')


#no_kids_share = data['status_child'].value_counts().loc['Домохозяйство без детей с долгом'] / data['status_child'].value_counts().loc['Домохозяйство без детей и без долга']
#print('Доля не возврата долга в категории:' + ' Нет детей ' + '{:.2%}'.format(no_kids_share))
#many_kids_share = data['status_child'].value_counts().loc['Многодетное домохозяйство с долгом'] / data['status_child'].value_counts().loc['Многодетное домохозяйство без долга']
#print('Доля не возврата долга в категории:' + ' Многодетное домохозяйство ' + '{:.2%}'.format(many_kids_share))
#not_many_kids_share = data['status_child'].value_counts().loc['Нуклеарное домохозяйство с долгом'] / data['status_child'].value_counts().loc['Нуклеарное домохозяйство без долгов']
#print('Доля не возврата долга в категории:' + ' Нуклеарное домохозяйство ' + '{:.2%}'.format(not_many_kids_share))

data.groupby('children_status').debt.agg(['count','mean'])


Домохозяйство без детей и без долга    13044
Нуклеарное домохозяйство без долгов     4411
Многодетное домохозяйство без долга     2275
Домохозяйство без детей с долгом        1063
Нуклеарное домохозяйство с долгом        445
Многодетное домохозяйство с долгом       233
Name: status_child, dtype: int64
--------------------------------
['Нуклеарное домохозяйство без долгов'
 'Домохозяйство без детей и без долга'
 'Многодетное домохозяйство без долга' 'Домохозяйство без детей с долгом'
 'Нуклеарное домохозяйство с долгом' 'Многодетное домохозяйство с долгом']
--------------------------------


,count,mean
children_status,,
Многодетное домохозяйство,2508,0.092903
Нет детей,14107,0.075353
Нуклеарное домохозяйство,4856,0.091639


**Вывод**

Удивительно, но семьи с детьми чаще остальных не возвращают долг вовремя. Самые дисциплинированыне домохозяйства без детей. Исходя из показателей чем больше детей в семье тем чаще домохозяйство не возвращает долг

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
def family_debt(row):
    
    family_status_cat = row['family_status_cat']
    debt = row['debt']
    
    if family_status_cat == 'Одинокие':
        if debt == 1:
                return 'Одинокие с долгом'
        return 'Одинокие без долга'
    if family_status_cat == 'Семейные':
        if debt == 1:
                return 'Семейные с долгом'
    return 'Семейные без долга'

row_values = [data['family_status_cat'], data['debt']]
row_columns = ['family_status_cat', 'debt']
row = pd.Series(data=row_values , index=row_columns) 
data['family_deb_status'] = data.apply(family_debt, axis=1)

print(data['family_deb_status'].value_counts())
print('--------------------------------')
print(data['family_deb_status'].unique())
print('--------------------------------')

#single_share = data['family_deb_status'].value_counts().loc['Одинокие с долгом'] / data['family_deb_status'].value_counts().loc['Одинокие без долга']
#print('Доля не возврата долга в категории:' + ' Одинокие ' + '{:.2%}'.format(single_share))
#family_share = data['family_deb_status'].value_counts().loc['Семейные с долгом'] / data['family_deb_status'].value_counts().loc['Семейные без долга']
#print('Доля не возврата долга в категории:' + ' Семейные ' + '{:.2%}'.format(family_share))

data.groupby('family_status_cat').debt.agg(['count','mean'])

Семейные без долга    15188
Одинокие без долга     4542
Семейные с долгом      1319
Одинокие с долгом       422
Name: family_deb_status, dtype: int64
--------------------------------
['Семейные без долга' 'Семейные с долгом' 'Одинокие без долга'
 'Одинокие с долгом']
--------------------------------


,count,mean
family_status_cat,,
Одинокие,4964,0.085012
Семейные,16507,0.079905


**Вывод**

В то же время семейный люди более дисциплинированы

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
def income_debt(row):
    
    income_status = row['income_status']
    debt = row['debt']
    
    if income_status == 'Бедные':
        if debt == 1:
                return 'Бедные с долгом'
        return 'Бедные без долга'
    if income_status == 'Очень богатые':
        if debt == 1:
                return 'Очень богатые с долгом'
        return 'Очень богатые без долга'
    if income_status == 'Богатые':
        if debt == 1:
                return 'Богатые с долгом'
            
    return 'Богатые без долгов'

row_values = [data['income_status'], data['debt']]
row_columns = ['income_status', 'debt']
row = pd.Series(data=row_values , index=row_columns) 
data['income_deb_status'] = data.apply(income_debt, axis=1)

print(data['income_deb_status'].value_counts())
print('--------------------------------')
print(data['income_deb_status'].unique())
print('--------------------------------')

#poor_share = data['income_deb_status'].value_counts().loc['Бедные с долгом'] / data['income_deb_status'].value_counts().loc['Бедные без долга']
#print('Доля не возврата долга в категории:' + ' До ста тысяч ' + '{:.2%}'.format(poor_share))
#king_share = data['income_deb_status'].value_counts().loc['Очень богатые с долгом'] / data['income_deb_status'].value_counts().loc['Очень богатые без долга']
#print('Доля не возврата долга в категории:' + ' Свыше полумилиона ' + '{:.2%}'.format(king_share))
#rich_share = data['income_deb_status'].value_counts().loc['Богатые с долгом'] / data['income_deb_status'].value_counts().loc['Богатые без долгов']
#print('Доля не возврата долга в категории:' + ' Остальные ' + '{:.2%}'.format(rich_share))

data.groupby('income_status').debt.agg(['count','mean'])


Богатые без долгов         15413
Бедные без долга            4109
Богатые с долгом            1373
Бедные с долгом              354
Очень богатые без долга      208
Очень богатые с долгом        14
Name: income_deb_status, dtype: int64
--------------------------------
['Богатые без долгов' 'Бедные без долга' 'Богатые с долгом'
 'Очень богатые без долга' 'Бедные с долгом' 'Очень богатые с долгом']
--------------------------------


,count,mean
income_status,,
Бедные,4463,0.079319
Богатые,16786,0.081794
Очень богатые,222,0.063063


**Вывод**

Очень богатые возвращают долги лучше всех. Бедные чуть хуже, но все же лучше чем Богатые. 

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
def purpose_debt(row):
    
    cat = row['category']
    debt = row['debt']
    
    if cat == 'недвижимость':
        if debt == 1:
                return 'С квартирой с долгом'
        return 'С квартирой без долга'
    if cat == 'автомобиль':
        if debt == 1:
                return 'С автомобилем с долгом'
        return 'С автомобилем без долга'
    if cat == 'образование':
        if debt == 1:
                return 'С образованием с долгом'
        return 'С образованием без долга'    
    if cat == 'свадьба':
        if debt == 1:
                return 'Женатые с долгом'       
            
    return 'Женатые с без долгов'

row_values = [data['category'], data['debt']]
row_columns = ['category', 'debt']
row = pd.Series(data=row_values , index=row_columns) 
data['purpose_deb_status'] = data.apply(purpose_debt, axis=1)

print(data['purpose_deb_status'].value_counts())
print('--------------------------------')
print(data['purpose_deb_status'].unique())
print('--------------------------------')

#flat_share = data['purpose_deb_status'].value_counts().loc['С квартирой с долгом'] / data['purpose_deb_status'].value_counts().loc['С квартирой без долга']
#print('Доля не возврата долга в категории:' + ' недвижимость ' + '{:.2%}'.format(flat_share))
#car_kids_share = data['purpose_deb_status'].value_counts().loc['С автомобилем с долгом'] / data['purpose_deb_status'].value_counts().loc['С автомобилем без долга']
#print('Доля не возврата долга в категории:' + ' автомобиль ' + '{:.2%}'.format(car_kids_share))
#edu_share = data['purpose_deb_status'].value_counts().loc['С образованием с долгом'] / data['purpose_deb_status'].value_counts().loc['С автомобилем без долга']
#print('Доля не возврата долга в категории:' + ' образование ' + '{:.2%}'.format(edu_share))
#mar_share = data['purpose_deb_status'].value_counts().loc['Женатые с долгом'] / data['purpose_deb_status'].value_counts().loc['Женатые с без долгов']
#print('Доля не возврата долга в категории:' + ' свадьба ' + '{:.2%}'.format(mar_share))

data.groupby('category').debt.agg(['count','mean'])

С квартирой без долга       10032
С автомобилем без долга      3905
С образованием без долга     3644
Женатые с без долгов         2149
С квартирой с долгом          782
С автомобилем с долгом        403
С образованием с долгом       370
Женатые с долгом              186
Name: purpose_deb_status, dtype: int64
--------------------------------
['С квартирой без долга' 'С автомобилем без долга'
 'С образованием без долга' 'Женатые с без долгов' 'С квартирой с долгом'
 'Женатые с долгом' 'С образованием с долгом' 'С автомобилем с долгом']
--------------------------------


,count,mean
category,,
автомобиль,4308,0.093547
недвижимость,10814,0.072314
образование,4014,0.092177
свадьба,2335,0.079657


**Вывод**

Очевидно долг за недвижимость дисциплинирует людей. А вот кредит на автомобиль люди не спешат возвращать.

## Шаг 4. Общий вывод

In [19]:
#print('Доля не возврата долга в категории:' + ' Нет детей ' + '{:.2%}'.format(no_kids_share))

#print('Доля не возврата долга в категории:' + ' Многодетное домохозяйство ' + '{:.2%}'.format(many_kids_share))

#print('Доля не возврата долга в категории:' + ' Нуклеарное домохозяйство ' + '{:.2%}'.format(not_many_kids_share))

#print('Доля не возврата долга в категории:' + ' Одинокие ' + '{:.2%}'.format(single_share))

#print('Доля не возврата долга в категории:' + ' Семейные ' + '{:.2%}'.format(family_share))

#print('Доля не возврата долга в категории:' + ' До ста тысяч ' + '{:.2%}'.format(poor_share))

#print('Доля не возврата долга в категории:' + ' Свыше полумилиона ' + '{:.2%}'.format(king_share))

#print('Доля не возврата долга в категории:' + ' Остальные ' + '{:.2%}'.format(rich_share))

#print('Доля не возврата долга в категории:' + ' недвижимость ' + '{:.2%}'.format(flat_share))

#print('Доля не возврата долга в категории:' + ' автомобиль ' + '{:.2%}'.format(car_kids_share))

#print('Доля не возврата долга в категории:' + ' образование ' + '{:.2%}'.format(edu_share))

#print('Доля не возврата долга в категории:' + ' свадьба ' + '{:.2%}'.format(mar_share))

print(data.groupby('children_status').debt.agg(['count','mean']))
print(data.groupby('family_status_cat').debt.agg(['count','mean']))
print(data.groupby('income_status').debt.agg(['count','mean']))
print(data.groupby('category').debt.agg(['count','mean']))

                           count      mean
children_status                           
Многодетное домохозяйство   2508  0.092903
Нет детей                  14107  0.075353
Нуклеарное домохозяйство    4856  0.091639
                   count      mean
family_status_cat                 
Одинокие            4964  0.085012
Семейные           16507  0.079905
               count      mean
income_status                 
Бедные          4463  0.079319
Богатые        16786  0.081794
Очень богатые    222  0.063063
              count      mean
category                     
автомобиль     4308  0.093547
недвижимость  10814  0.072314
образование    4014  0.092177
свадьба        2335  0.079657


Лучше всего долги возвращают бездетные семейные домохозяйства с доходом выше 500 тысяч взявшие кредит на недвижимость.
Хуже всего возвращают долги Семейные многодетные домохозяйства, с доходом 100-499 тысяч и взявшие кредит на автомобиль.

Профиль самого частого заемщика - семейные люди с доходом 100-499 тысяч без детей с целью кредита на недвижимость. Такие заемщики попадают в категорию лучших по трем показателям из четырех. Хороший вариант дать в долг.

Однако изменение семейного статуса и появление детей сказывается негативно на возвращаемость долгов вовремя.

А кредит на автомобиль является самым рискованных кейсом в принципе.
